# Work with Azure Blob Storage

To start, ensure that you have a Azure Blob Storage account. Reference the documentation on how to create a storage account if needed. [Create a storage account](https://learn.microsoft.com/azure/storage/common/storage-account-create?tabs=azure-portal)

Also, have a container created in the Azure Blob Storage account. You can use this quick-start to help guide you in creating a container. [Quickstart: Upload, download, and list blobs with the Azure portal](https://learn.microsoft.com/azure/storage/blobs/storage-quickstart-blobs-portal)

Blob Storage REST endpoint are documented here: [Azure Blob Storage REST API](https://learn.microsoft.com/rest/api/storageservices/blob-service-rest-api). 
The samples use the [Get Blob](https://learn.microsoft.com/rest/api/storageservices/get-blob?tabs=azure-ad), [Put Blob](https://learn.microsoft.com/en-us/rest/api/storageservices/put-blob?tabs=azure-ad), and the (Create Container)[https://learn.microsoft.com/rest/api/storageservices/create-container?tabs=azure-ad] APIs.

The following samples assume that there is an Azure Blob Storage Account deployed at `https://blobby.blob.core.windows.net/` and a container named `myblobs`. To have the samples working in your environment, make sure adjust URL and container name so that they will match your account.

## Work with files in Azure Blob Storage using a SAS Token

Only authenticated requests can send REST requests to Azure Blob Storage. One way to authenticate a request is to provide a Shared Access Signature token: 
- [Delegate access by using a shared access signature](https://learn.microsoft.com/rest/api/storageservices/delegate-access-with-shared-access-signature)
- [Create an account SAS](https://learn.microsoft.com/rest/api/storageservices/create-account-sas). 

At the moment is not possible to generate a SAS token directly from Azure SQL database, but you can put the code for generating such a token in an Azure Function and call it from Azure SQL database using `sp_invoke_external_rest_point` as well.

Once you have the token you can add it into a Database Scoped Credential:

In [ ]:
-- make sure a database master key exists
if not exists(select * from sys.symmetric_keys where [name] = '##MS_DatabaseMasterKey##') begin
    create master key encryption by password = 'LONg_Pa$$_w0rd!'
end

-- create database scoped credential
create database scoped credential [filestore]
with identity='SHARED ACCESS SIGNATURE', 
secret='sv=2022-11-02&ss=bfqt&srt=sco&sp=seespotrun&se=2023-08-03T02:21:25Z&st=2023-08-02T18:21:25Z&spr=https&sig=WWwwWWwwWWYaKCheeseNXCCCCCCDDDDDSSSSSU%3D'
go

You can now create a file and add content to the file:

In [ ]:
declare @payload nvarchar(max) = (select * from (values('Hello from Azure SQL!', sysdatetime())) payload([message], [timestamp])for json auto, without_array_wrapper)
declare @response nvarchar(max), @url nvarchar(max), @headers nvarchar(1000);
declare @len int = len(@payload)

-- Create the File
set @url = 'https://blobby.blob.core.windows.net/myblobs/test-me-from-azure-sql.json'
set @headers = json_object(
        'x-ms-type': 'file',
        'x-ms-content-length': cast(@len as varchar(9)),
        'Accept': 'application/xml')
exec sp_invoke_external_rest_endpoint
    @url = @url,
    @method = 'PUT',
    @headers = @headers,
    @credential = [filestore],
    @response = @response output
select cast(@response as xml);

-- Add text to the File
set @headers = json_object(
        'x-ms-range': 'bytes=0-' + cast(@len-1 as varchar(9)),
        'x-ms-write': 'update',
        'Accept': 'application/xml');
set @url = 'https://blobby.blob.core.windows.net/myblobs/test-me-from-azure-sql.json'
set @url += '?comp=range'
exec sp_invoke_external_rest_endpoint
    @url = @url,
    @method = 'PUT',
    @headers = @headers,
    @payload = @payload,
    @credential = [filestore],
    @response = @response output
select cast(@response as xml)
go

Now, using the Get Blob REST endpoint, you can read the contents of the file:

In [ ]:
declare @response nvarchar(max);
declare @url nvarchar(max) = 'https://blobby.blob.core.windows.net/myblobs/test-me-from-azure-sql.json'
exec sp_invoke_external_rest_endpoint
    @url = @url,
    @headers = '{"Accept":"application/xml"}',
    @credential = [filestore],
    @method = 'GET',
    @response = @response output
select cast(@response as xml)
go

## Work with files in Azure Blob Storage using Managed Identities

First, follow the instructions here: [Enable Managed Identity in Azure SQL](./azure-sql-enable-msi.ipynb) to enable Managed Identity for your Azure SQL database. Next, add the Azure SQL Database to the Storage Blob Data Owner role. This is done via Access Control (IAM) in the Azure Portal or via Azure CLI.
To start, navigate to the container page and click **+ Add**. Then in the dropdown, click **Add role assignment**.

![A picture of adding a role assignment on the Access Control page for a container](./assets/files-add-roll.png)

On the following page, use the search box and enter **blob owner**. The roles list should then show the **Storage Blob Data Owner** role. Click the role and then click the **Next** button on the bottom of the page.

![A picture of selecting the Storage Blob Data Owner role and then clicking Next](./assets/files-add-roll2.png)

On the next page, click the **Managed identity** radio button then click the **+ Select members** link.

![A picture of clicking the Managed identity radio button then clicking the + Select members link](./assets/files-add-roll3.png)

In the slide out blade on the right, use the **Subscription** drop down to select the subscription where your Azure SQL Database is located. Then use the **Managed identity** dropdown to select **SQL server**. Use the **Select** field to filter the databases by name.

![A picture of using the Subscription drop down to select the subscription where your Azure SQL Database is located then using the Managed identity dropdown to select SQL server](./assets/files-add-roll4.png)

Once you find the database that is using the REST Endpoint Invocation feature from, select it. It then appears as a selected managed identity. Now, click the **Select** button on the bottom left of the blade.

![A picture of clicking the select button on the bottom left of the blade after a database managed identity has been selected](./assets/files-add-roll5.png)

Back on the Add role assignments page, you will see the selected database managed identity. Click the blue **Review + assign** button in the lower left of the page.

![A picture of viewing the selected database managed identity on the page and then clicking the blue **Review + assign** button in the lower left of the page](./assets/files-add-roll6.png)

On the final page, again click the **Review + assign** button in the lower left of the page.

![A picture of clicking the **Review + assign** button in the lower left of the page](./assets/files-add-roll7.png)


 , and then check how to grant to right permission on Event Hubs to the Azure SQL Manage Identity, following the instructions here: [Grant permissions to a managed identity in Azure AD](https://learn.microsoft.com/azure/event-hubs/authenticate-managed-identity?tabs=latest#grant-permissions-to-a-managed-identity-in-azure-ad).

Once that is done you just need to create a Database Scoped Credentials with the string `Managed Identity` as identity and `https://eventhubs.azure.net` as the `resourceid`:

In [ ]:
-- make sure a database master key exists
if not exists(select * from sys.symmetric_keys where [name] = '##MS_DatabaseMasterKey##') begin
    create master key encryption by password = 'LONg_Pa$$_w0rd!'
end

-- create database scoped credential
if exists(select * from sys.database_scoped_credentials where [name] = 'https://azure-event-hubs.servicebus.windows.net') begin
    drop database scoped credential [https://azure-event-hubs.servicebus.windows.net];
end;
create database scoped credential [https://azure-event-hubs.servicebus.windows.net]
with identity = 'Managed Identity', 
secret = '{"resourceid": "https://eventhubs.azure.net" }';

Once this is done you can send the message using the same code as before:

In [ ]:
declare @payload nvarchar(max) = '{"UserId": "6C5E29A2-A5E7-449D-BD14-259D61ADC6BE", "FirstName": "John", "LastName": "Doe"}';
declare @headers nvarchar(4000) = N'{"BrokerProperties": "' + string_escape('{"PartitionKey": "6C5E29A2-A5E7-449D-BD14-259D61ADC6BE"}', 'json') + '"}'
declare @ret int, @response nvarchar(max)

exec @ret = sp_invoke_external_rest_endpoint 
        @url = 'https://azure-event-hubs.servicebus.windows.net/myeventhub/messages',
        @headers = @headers,
        @payload = @payload,
		@credential = [https://azure-event-hubs.servicebus.windows.net],
        @response = @response output;

select @response;